In [1]:
import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
model = EfficientPhys(TN=True)
model.compile(optimizer='adamw', loss='mse')
model(np.random.random((4, 160, 72, 72, 3)));
model.summary()

Model: "efficient_phys"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tsm (TSM)                       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (640, 72, 72, 32)      │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (640, 70, 70, 32)      │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (640, 35, 35, 64)      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (640, 33, 33, 64)      │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (640, 70, 70, 1)       │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_mask (Attention_mask) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (640, 33, 33, 1)       │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (640, 35, 35, 32)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (640, 35, 35, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (640, 128)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (640, 128)             │     2,097,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (640, 1)               │           129 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (640, 5184, 3)         │        20,736 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (640, 16384)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,183,811 (8.33 MB)

 Trainable params: 2,173,443 (8.29 MB)

 Non-trainable params: 10,368 (40.50 KB)

In [3]:
batch_size = 32

tape = "/root/ssd_cache/rppg_training_data/mmpd_160x72x72_all"

train_tape = load_datatape(tape, fold='train', batch=batch_size)
valid_tape = load_datatape(tape, fold='val', extended_hr='False', batch=batch_size)

#train_rlap = DatatapeMonitor(train_tape)
training_set, validation_set = KerasDataset(train_tape), KerasDataset(valid_tape)

training_set = training_set.apply_fn(compress_aug)

mmpd_160x72x72_all             datatape has been loaded.     7648 items total. fold=train selected
mmpd_160x72x72_all             datatape has been loaded.     1664 items total. fold=val&extended_hr=False selected


In [4]:
#model.fit(training_set, validation_data=validation_set, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/efficientphystn_mmpd.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(model, training_set, validation_set, epochs=20, check_point_path='../weights/efficientphystn_mmpd.weights.h5')
model.load_weights('../weights/efficientphystn_mmpd.weights.h5')

Training:   0%|          | 0/239 [00:00<?, ?it/s]

Validating:   0%|          | 0/52 [00:00<?, ?it/s]

Epoch   1:  Training loss: 1.017	Validation loss:0.9227	


Training:   0%|          | 0/239 [00:00<?, ?it/s]

Validating:   0%|          | 0/52 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/52 [00:00<?, ?it/s]

Epoch  19:  Training loss:0.8222	Validation loss: 0.909	


Training:   0%|          | 0/239 [00:00<?, ?it/s]

Validating:   0%|          | 0/52 [00:00<?, ?it/s]

Epoch  20:  Training loss:0.8203	Validation loss:0.9082	


In [5]:
eval_on_dataset(dataset_H5_rlap, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhystn_MMPD_RLAP.h5', scenes=['R1', 'R2', 'R3', 'R4'])
get_metrics('../results/EfficientPhystn_MMPD_RLAP.h5', dropped='False')

  0%|          | 0/220 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,4.425±0.149,11.48±2.777,5.849±0.206,0.61029
Whole,2.379±0.254,4.45±2.064,3.119±0.336,0.90447


In [6]:
eval_on_dataset(dataset_H5_cohface, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhystn_MMPD_COHFACE.h5', fps=30)
get_metrics('../results/EfficientPhystn_MMPD_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,8.28±0.336,16.565±4.206,11.396±0.466,0.37182
Whole,5.716±0.61,9.684±4.57,7.731±0.763,0.62398


In [7]:
eval_on_dataset(dataset_H5_pure, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhystn_MMPD_PURE.h5')
get_metrics('../results/EfficientPhystn_MMPD_PURE.h5')

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,5.441±0.553,16.194±6.444,7.343±0.715,0.74811
Whole,3.864±1.06,9.01±6.083,5.005±1.245,0.92647


In [8]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhystn_MMPD_UBFC.h5')
get_metrics('../results/EfficientPhystn_MMPD_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,1.91±0.169,4.309±2.379,2.022±0.19,0.96835
Whole,0.9±0.23,1.738±1.403,0.962±0.263,0.99458
